<a href="https://colab.research.google.com/github/anjosmelanie/cd0281-Introduction-to-Neural-Networks-with-PyTorch/blob/master/Dermatologist_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import numpy as np

# Dataset
from torchvision import datasets, transforms, models 
from torch.utils.data import DataLoader, SubsetRandomSampler

# Training Model
from torch import nn
from torch import optim
from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F

# Matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# Extra Information
from PIL import Image
import tqdm 
from torchsummary import summary


In [ ]:
## Download datasets, once per session only
import urllib

train_url = 'https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/skin-cancer/train.zip'
valid_url = 'https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/skin-cancer/valid.zip'
test_url = 'https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/skin-cancer/test.zip'

def download(url, zip):
  urllib.request.urlretrieve(url, zip)
  print(zip + ' downloaded successfully!')

download(train_url, 'train.zip')
! unzip 'train.zip'

download(valid_url, 'valid.zip')
! unzip 'valid.zip'

download(test_url, 'test.zip')
! unzip 'test.zip'


train.zip downloaded successfully!
Archive:  train.zip
replace train/melanoma/ISIC_0010034.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
## TODO: Eventually mix up traning + valid data and split them

train_path = "train"
valid_path = "valid"
test_path = "test"

image_size = 224
batch_size = 20
valid_percent = 0.2

mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

# ------------------------------------------------ #

# Data transform
transform_train = transforms.Compose([
    transforms.RandomRotation(45),
    transforms.RandomResizedCrop(image_size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

transform_test = transforms.Compose([
    transforms.RandomResizedCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

# ------------------------------------------------ #

# Dataset
# Load and transform data using ImageFolder
train_data = datasets.ImageFolder(train_path, transform = transform_test)
print('Number of training images: ', len(train_data))

valid_data = datasets.ImageFolder(valid_path, transform = transform_test)
print('Number of validation images: ', len(train_data))

test_data = datasets.ImageFolder(test_path, transform = transform_test)
print('Number of test images: ', len(test_data))

# ------------------------------------------------ #

# Prepare data loaders
train_data_loader = DataLoader(train_data, batch_size=batch_size)
valid_data_loader = DataLoader(valid_data, batch_size=batch_size)
test_data_loader = DataLoader(test_data, batch_size=batch_size)

# ------------------------------------------------ #

loaders = {'train': train_data_loader, 'valid': valid_data_loader, 'test': test_data_loader}


In [ ]:
classes_raw = train_data.classes
#classes = [' '.join(_c[3:].split('_')) for _c in classes_raw] # Label formatting (remove number and replace '_' for spaces)
num_classes = len(classes_raw)

# Visualize one batch
img, label = next(iter(train_data_loader))
fig = plt.figure(figsize=(16,12))

for i in range(batch_size):
    single_image = img[i]
    label_axis = plt.subplot(4, 5, i + 1)
    single_image = single_image * 0.5 + 0.5
    plt.imshow(np.transpose(single_image, (1,2,0)))
    label_axis.set_title(classes_raw[label[i]])
    label_axis.axis('off')

In [ ]:
# useful variable that tells us whether we should use the GPU
use_cuda = torch.cuda.is_available()
print(use_cuda)

In [ ]:
## Loss Function
criterion = nn.CrossEntropyLoss()

## Optimizer
def get_optimizer(model):
      params_to_update = []
      for name, param in model.named_parameters():
          if param.requires_grad == True:
              params_to_update.append(param)
      
      return optim.Adam(params_to_update, lr=0.00015) 

In [ ]:
## Architecture

def get_transfered_model_resnet50():

    model_transfer = models.resnet50(pretrained=True)
    
    for param in model_transfer.parameters():
        param.requires_grad = False

    model_transfer.fc = nn.Linear(2048, num_classes)
    
    optimizer = get_optimizer(model_transfer)
    
    return model_transfer, optimizer

## Specify model architecture

model_transfer, optimizer = get_transfered_model_resnet50()

print(model_transfer)

In [ ]:
## Train the model and save the best model parameters at filepath 'model_transfer.pt'

def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    # scheduler to decrease the learning rate
    scheduler = StepLR(optimizer, step_size=10, gamma=0.1)
    
    # ------------------------------------------------ #
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        # set the module to training mode
        model.train()
        for batch_idx, (data, target) in enumerate(tqdm.tqdm(loaders['train'])):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()

            ## find the loss and update the model parameters accordingly
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            # Update the train_loss
            train_loss = 1./(batch_idx+1) * (batch_idx*train_loss + loss.data.item())
            
        scheduler.step()

        # set the model to evaluation mode
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()

            ## update average validation loss 
            output = model(data)
            loss = criterion(output, target)
            valid_loss = 1./(batch_idx+1) * (batch_idx*valid_loss + loss.data.item())


        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))

        ## if the validation loss has decreased, save the model at the filepath stored in save_path
        if valid_loss < valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min, valid_loss))
            valid_loss_min = valid_loss
            valid_loss_min_epoch = epoch
            torch.save(model.state_dict(), save_path)   
            
        # ------------------------------------------------ #
            
        # Early Stopping    
            
        if early_stopping(epoch, valid_loss, valid_loss_min, valid_loss_min_epoch, 5, 0.05):
            break
        else:
            pass
                
        # ------------------------------------------------ #
        
    return model

    # ------------------------------------------------ #
    
def early_stopping(epoch, valid_loss, valid_loss_min, valid_loss_min_epoch, patience, stop_th):    
        if (epoch - valid_loss_min_epoch) > patience:
            # Stop the training if the current loss is greater than the threshold, or just keep training 
            if (valid_loss - valid_loss_min) > (stop_th * valid_loss_min): 
                print("The validation loss jumped across the threshold, stop.")
                return True
            else:
                return False    

    # ------------------------------------------------ #
    
def transfer_weight_init(m):
## implement a weight initialization strategy
    class_name = m.__class__.__name__
    if class_name.find("Linear") != -1:
        n = m.in_features
        y = 1.0/np.sqrt(n)
        m.weight.data.normal_(0, y)
        m.bias.data.fill_(0)
        
    # ------------------------------------------------ #
    
num_epochs = 25

model_transfer.apply(transfer_weight_init)

model_transfer = train(num_epochs, loaders, model_transfer, get_optimizer(model_transfer), criterion, use_cuda, 'model_transfer.pt') 
    
# load the model that got the best validation accuracy
model_transfer.load_state_dict(torch.load('model_transfer.pt'))


In [ ]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    # set the module to evaluation mode
    model.eval()

    for batch_idx, (data, target) in enumerate(tqdm.tqdm(loaders['test'])):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data.item() - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

# load the model that got the best validation accuracy
model_transfer.load_state_dict(torch.load('model_transfer.pt'))
test(loaders, model_transfer, criterion, use_cuda)

In [ ]:
def predict_diagnosis(img_path):
     
    # Open and transform image
    image_size = 224
    normalize = transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    transform_image = transforms.Compose([transforms.Resize((image_size,image_size)), transforms.ToTensor(), normalize])
    
    raw_image = Image.open(img_path)
    transformed_image = transform_image(raw_image)
    transformed_image = transformed_image.unsqueeze(0)
    if use_cuda:
        transformed_image = transformed_image.cuda()  
    
    # Load model and make a prediction
    model_transfer.load_state_dict(torch.load('model_transfer.pt'))
    model_transfer.eval()
    output = model_transfer(transformed_image)

    probs = torch.nn.functional.softmax(output, dim=1)
    classes = list(train_data.class_to_idx.keys())

    #for i in range(0,3):
    #  print("Probability of", classes[i], ": ", probs[0][i].item())

    prob_melanoma =  probs[0][0].item()
    prob_novus =  probs[0][1].item()
    prob_keratosis =  probs[0][2].item()

    #print(img_path, prob_melanoma, prob_keratosis)

    return img_path, prob_melanoma, prob_keratosis

# Testing Predictions
image_path1 = 'test/seborrheic_keratosis/ISIC_0012974.jpg'
image_path2 = 'test/nevus/ISIC_0015631.jpg'
print(predict_diagnosis(image_path1))
print(predict_diagnosis(image_path2))

In [ ]:
import csv

header = ['Id', 'task_1', 'task_2']

data = []
for i in range(0,600):
  data.append(predict_diagnosis(test_data.imgs[i][0]))

with open('predictions.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

     # write multiple rows
    writer.writerows(data)

# close the file
f.close()